In [2]:
import requests
import re
from bs4 import BeautifulSoup
import json
import gzip
from tqdm import tqdm
import pandas as pd
import numpy as np

In [2]:
body = requests.get('https://www.bbc.com/').text

In [3]:
soup = BeautifulSoup(body, 'html.parser')

In [4]:
a = soup.get_text(' ').replace('\n', '').lower()

In [5]:
b = re.sub(r"[^a-z ]+","", a)

In [6]:
c = ' '.join(b.split())

In [13]:
reader = pd.read_json("../data/websites_1000_5cat_html.json.gz", orient='records', lines=True, chunksize=100)

In [14]:
df_main = pd.DataFrame([])

for chunk in reader:
    df_chunk = pd.DataFrame(chunk)
    df_chunk = create_w2v_avg(df_chunk)
    df_main = pd.concat((df_main, df_chunk))

TypeError: object of type 'NoneType' has no len()

In [24]:
df = pd.read_json("../data/websites_1000_5cat_html.json.gz", orient='records', lines=True)

In [27]:
valid_df(df)

,url,html,cat
1,http://www.allwatchers.com/topics/info_4024.asp,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01//...",Arts
2,http://www.opmden.com/,"<!DOCTYPE HTML PUBLIC ""-//IETF//DTD HTML 2.0//...",Arts
3,http://www.angelfire.com/ns/phil/,<!DOCTYPE HTML>\n\n<html>\n<head>\n<meta http-...,Arts
4,http://www.beatricegormley.com,<!DOCTYPE html>\n<html>\n<head>\n<meta content...,Arts
5,http://vmnagaraj.tripod.com/,"<HTML>\n\n<HEAD>\n<TOBJ bgimage=""/imagelib/sty...",Arts
...,...,...,...
4995,http://www.space-explorers.com/,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01//...",Science
4996,http://cse.nias.affrc.go.jp/shignak/chromosome...,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.01 T...",Science
4997,http://www.eea.europa.eu/,\n\n\n\n\n\n<!DOCTYPE html>\n\n<!-- paulirish....,Science
4998,http://www.goes.noaa.gov/index.html,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T...",Science


In [6]:
df_valid = df[df.apply(lambda x: is_valid(x.html), axis=1)]

In [ ]:
create_w2v_avg(df)

In [30]:
def create_w2v_avg(df):
    df_valid = clean_df(df)
    df_valid['emb'] = df_valid.apply(lambda x: word2vec_avg(clean_html(x.html)), axis=1)
    return df[['emb', 'cat0']]

In [31]:
def clean_df(df):
    df_valid = df[df.html.notnull()]
    df_valid = df_valid[df_valid.apply(lambda x: x.html != '', axis=1)]
    return df_valid

In [32]:
def clean_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    a = soup.get_text(' ').replace('\n', '').lower()
    b = re.sub(r"[^a-z ]+","", a)
    c = ' '.join(b.split())
    return c

In [34]:
import gensim.downloader as api
model = api.load('word2vec-google-news-300')

def word2vec_avg(str):
    
    acc = np.zeros(300)
    counter = 0
    
    for word in str.split(' '):
        try:
            vector = model[word]
            acc += vector
            counter += 1
        except:
            pass
    if counter != 0:
        return acc / counter
    return None

SyntaxError: invalid syntax (<ipython-input-34-4d848f3e91b7>, line 16)

In [5]:
df2 = pd.read_csv('test.gz', index_col=0)

In [6]:
df2

,emb,cat
1,[-7.25809733e-03 -4.13004557e-02 4.87467448e-...,Arts
2,[ 0.06930542 -0.00147448 0.05805086 0.025763...,Arts
3,[ 0.02900294 0.02974982 0.00379382 0.100300...,Arts
4,[ 0.06388844 0.02235351 0.00715072 0.120377...,Arts
5,[ 4.05956437e-02 3.02329695e-02 6.02138744e-...,Arts
...,...,...
4995,[-0.2590332 -0.06420898 -0.13989258 0.047531...,Science
4996,[ 2.20493224e-02 -1.80053711e-02 4.51764363e-...,Science
4997,[-2.56318912e-02 3.69664736e-02 3.30749189e-...,Science
4998,[-0.03867929 -0.00724498 -0.00488065 0.017599...,Science
